In [2]:
import pandas as pd
import numpy as np
import us
from geopy.geocoders import Nominatim
from sklearn.neighbors import NearestNeighbors

In [3]:
demographicData = pd.read_csv("../data/Demographics/ACSDP1Y2019.csv", skiprows=[0])
incomeData = pd.read_csv("../data/Income/ACSST1Y2019.csv", skiprows=[0])
urbanData = pd.read_csv("2010CensusUrbanRural.csv", skiprows=[0])


In [4]:
demographicData

,id,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population!!Male,Percent!!SEX AND AGE!!Total population!!Male,Percent Margin of Error!!SEX AND AGE!!Total population!!Male,...,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female"
0,0400000US01,Alabama,4903185,*****,4903185,(X),2369611,5813,48.3,0.1,...,3731336,(X),1768386,4711,47.4,0.1,1962950,4023,52.6,0.1
1,0400000US02,Alaska,731545,*****,731545,(X),380433,2130,52.0,0.3,...,533151,(X),278479,2343,52.2,0.3,254672,2031,47.8,0.3
2,0400000US04,Arizona,7278717,*****,7278717,(X),3620935,2892,49.7,0.1,...,5137474,(X),2533931,10100,49.3,0.1,2603543,9762,50.7,0.1
3,0400000US05,Arkansas,3017804,*****,3017804,(X),1474705,4896,48.9,0.2,...,2235415,(X),1075762,4729,48.1,0.2,1159653,4437,51.9,0.2
4,0400000US06,California,39512223,*****,39512223,(X),19640794,6786,49.7,0.1,...,26032160,(X),12786049,32037,49.1,0.1,13246111,27222,50.9,0.1
5,0400000US08,Colorado,5758736,*****,5758736,(X),2904955,4852,50.4,0.1,...,4244210,(X),2128648,8907,50.2,0.1,2115562,6315,49.8,0.1
6,0400000US09,Connecticut,3565287,*****,3565287,(X),1743777,2405,48.9,0.1,...,2619474,(X),1253707,6831,47.9,0.2,1365767,6310,52.1,0.2
7,0400000US10,Delaware,973764,*****,973764,(X),470288,1336,48.3,0.1,...,725178,(X),344221,2358,47.5,0.2,380957,2594,52.5,0.2
8,0400000US11,District of Columbia,705749,*****,705749,(X),334355,513,47.4,0.1,...,536768,(X),249106,2526,46.4,0.4,287662,2811,53.6,0.4
9,0400000US12,Florida,21477737,*****,21477737,(X),10498005,7711,48.9,0.1,...,15507315,(X),7459575,23711,48.1,0.1,8047740,19367,51.9,0.1


In [5]:
margins = [column for column in demographicData.columns if "Margin" in column]

In [6]:
demographicData = demographicData.drop(columns = margins)

In [7]:
geolocator = Nominatim(user_agent = "MDST-election-prediction")

In [8]:
lat = {}
long = {}
for index, state in demographicData["Geographic Area Name"].iteritems():
    loc = geolocator.geocode(state)
    lat[state] = loc.latitude
    long[state] = loc.longitude

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming
Puerto Rico


In [9]:
demographicData["Longitude"] = demographicData["Geographic Area Name"].map(long)

In [10]:
demographicData["Latitude"] = demographicData["Geographic Area Name"].map(lat)

In [11]:
feat_columns = ["Estimate!!SEX AND AGE!!Total population!!Median age (years)",
                "Percent!!RACE!!Total population!!One race!!Black or African American",
               "Percent!!RACE!!Total population!!One race!!American Indian and Alaska Native",
               "Percent!!RACE!!Total population!!One race!!Asian",
               "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)",
               "Latitude",
               "Longitude"]

In [13]:
demoFeatData = demographicData[feat_columns]

In [35]:
educationData = pd.read_csv("../data/ACSST1Y2019.S1501_data_with_overlays_2020-10-22T140417.csv",skiprows=([0]))

In [37]:
margins = [column for column in educationData.columns if "Margin" in column]

In [38]:
educationData = educationData.drop(columns = margins)

In [42]:
edu_columns = ["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years",
               "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher",
               "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over",
               "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher",
               "Estimate!!Percent!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher"
              ]

In [43]:
educationData = educationData[edu_columns]

In [45]:
educationData['total_with_college'] = educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"] + educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher"]

/Users/coding/Documents/MDST/election-prediction/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
educationData['total_pop'] = educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years"] + educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over"]

/Users/coding/Documents/MDST/election-prediction/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
educationData['percent_with_degree'] = educationData['total_with_college'] / educationData['total_pop'] * 100

/Users/coding/Documents/MDST/election-prediction/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
edu_feats = ["percent_with_degree", "Estimate!!Percent!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher"]

In [54]:
eduFeatData = educationData[edu_feats]

demoFeatData

In [57]:
demofeat_array = demoFeatData.to_numpy()

In [58]:
edufeat_array = eduFeatData.to_numpy()

In [63]:
#rural

In [64]:
#religion

In [65]:
#unemployment

In [68]:
feat_array = np.concatenate((demofeat_array,edufeat_array), axis = 1)

In [69]:
feat_array

array([[ 3.94000000e+01,  2.69000000e+01,  5.00000000e-01,
         1.30000000e+00,  4.50000000e+00,  3.32588817e+01,
        -8.68295337e+01,  2.41082327e+01,  2.85000000e+01],
       [ 3.50000000e+01,  3.10000000e+00,  1.58000000e+01,
         6.00000000e+00,  7.20000000e+00,  6.44459613e+01,
        -1.49680909e+02,  2.70531138e+01,  3.57000000e+01],
       [ 3.83000000e+01,  4.70000000e+00,  4.60000000e+00,
         3.30000000e+00,  3.17000000e+01,  3.43953420e+01,
        -1.11763276e+02,  2.77012216e+01,  3.11000000e+01],
       [ 3.88000000e+01,  1.55000000e+01,  6.00000000e-01,
         1.50000000e+00,  7.70000000e+00,  3.52048883e+01,
        -9.24479108e+01,  2.15988325e+01,  2.43000000e+01],
       [ 3.70000000e+01,  5.80000000e+00,  8.00000000e-01,
         1.48000000e+01,  3.94000000e+01,  3.67014631e+01,
        -1.18755997e+02,  3.21865479e+01,  3.56000000e+01],
       [ 3.71000000e+01,  4.20000000e+00,  1.00000000e+00,
         3.30000000e+00,  2.18000000e+01,  3.872517

In [ ]:
feat_array = 

In [19]:
nbrs = NearestNeighbors().fit(feat_array)

In [20]:
distances, indices = nbrs.kneighbors(feat_array)

In [21]:
indices

array([[ 0, 40, 18, 10, 42],
       [ 1, 37, 12, 26, 44],
       [ 2, 28,  5,  4, 43],
       [ 3, 25, 42, 14, 17],
       [ 4, 28,  2, 31, 43],
       [ 5, 44, 16, 27,  2],
       [ 6, 39, 21, 32, 30],
       [ 7, 33, 46, 20, 40],
       [ 8, 20, 10, 24, 40],
       [ 9, 30, 32, 13, 33],
       [10, 40,  0, 20, 18],
       [11, 37,  4, 28,  1],
       [12, 50, 44, 37, 26],
       [13, 14,  3, 42, 22],
       [14, 17, 35, 22, 49],
       [15, 49, 23, 27, 25],
       [16, 27, 36, 15, 23],
       [17, 14, 35, 48, 25],
       [18, 24,  0, 10, 40],
       [19, 45, 29, 48, 21],
       [20,  7, 10, 40, 33],
       [21, 39,  6, 47, 38],
       [22, 35, 14, 38, 17],
       [23, 15, 49, 27, 25],
       [24, 18,  0, 10, 40],
       [25,  3, 14, 17, 15],
       [26, 50, 41, 34, 12],
       [27, 16, 15, 50, 23],
       [28,  2,  4,  5, 37],
       [29, 45, 19, 48, 21],
       [30, 32,  6, 39, 21],
       [31, 43,  2,  4, 28],
       [32, 30,  6, 39, 21],
       [33, 46,  7, 40, 42],
       [34, 41

In [23]:
demographicData['Abbreviation'] = demographicData["Geographic Area Name"].map(us.states.mapping('name', 'abbr'))

In [30]:
for state in indices:
    print(demographicData.iloc[state[0]]['Geographic Area Name'] + ': ' + demographicData.iloc[state[1]]['Geographic Area Name']
         + ', ' + demographicData.iloc[state[2]]['Geographic Area Name'] + ', ' + demographicData.iloc[state[3]]['Geographic Area Name'] + ', ' 
          + demographicData.iloc[state[4]]['Geographic Area Name'])

Alabama: South Carolina, Louisiana, Georgia, Tennessee
Alaska: Oregon, Idaho, Montana, Utah
Arizona: Nevada, Colorado, California, Texas
Arkansas: Missouri, Tennessee, Indiana, Kentucky
California: Nevada, Arizona, New Mexico, Texas
Colorado: Utah, Kansas, Nebraska, Arizona
Connecticut: Rhode Island, Massachusetts, New York, New Jersey
Delaware: North Carolina, Virginia, Maryland, South Carolina
District of Columbia: Maryland, Georgia, Mississippi, South Carolina
Florida: New Jersey, New York, Illinois, North Carolina
Georgia: South Carolina, Alabama, Maryland, Louisiana
Hawaii: Oregon, California, Nevada, Alaska
Idaho: Wyoming, Utah, Oregon, Montana
Illinois: Indiana, Arkansas, Tennessee, Michigan
Indiana: Kentucky, Ohio, Michigan, Wisconsin
Iowa: Wisconsin, Minnesota, Nebraska, Missouri
Kansas: Nebraska, Oklahoma, Iowa, Minnesota
Kentucky: Indiana, Ohio, West Virginia, Missouri
Louisiana: Mississippi, Alabama, Georgia, South Carolina
Maine: Vermont, New Hampshire, West Virginia, Mass